In [1]:
import pathlib
import os
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(pathlib.Path(os.path.dirname(os.path.abspath(""))).resolve()))
%load_ext autoreload
%autoreload 2

In [2]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import DESCNUpliftModelFactory
from src.models.NNUpliftModeling.DESCNUpliftModel import DESCNUpliftModel

/home/ogrobertino/test_env/lib/python3.10/site-packages/xgboost/core.py:377: FutureWarning: Your system has an old version of glibc (< 2.28). We will stop supporting Linux distros with glibc older than 2.28 after **May 31, 2025**. Please upgrade to a recent Linux distro (with glibc >= 2.28) to use future versions of XGBoost.
Note: You have installed the 'manylinux2014' variant of XGBoost. Certain features such as GPU algorithms or federated learning are not available. To use these features, please upgrade to a recent Linux distro with glibc 2.28+, and install the 'manylinux_2_28' variant.
  warnings.warn(
Failed to import duecredit due to No module named 'duecredit'


In [3]:
ds_name = 'lazada_v2'
features_percent = 100
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)

In [4]:
import pandas as pd
ds = pd.read_csv(train_path, sep='\t')

In [5]:
input_size = len(ds.columns) - 2
input_size

83

In [39]:
params = {
    'input_dim': input_size,             # Должно быть задано в соответствии с данными
    'share_dim': [32], # Варианты размерности общих слоев
    'base_dim': [32],   # Варианты размерности базовых слоев
    'do_rate': [0.1],  # Варианты dropout
    'batch_norm': True, # Использование BatchNorm
    'normalization': 'none', # Тип нормализации
    'prpsy_w': 0.5,   # Вес для потери пропенсити
    'escvr1_w': 0.5,  # Вес для потери ESTR
    'escvr0_w': 1.0,  # Вес для потери ESCR
    'mu1hat_w': 1.0,  # Вес для перекрестной потери TR
    'mu0hat_w': 0.5,  # Вес для перекрестной потери CR
    'batch_size': 32,
    'gradient_accumulation_steps' : 1,
    'optimizer': 
        {
            'name': 'Adam',
            'lr': 0.001,
            'weight_decay': 0.0001
        },
    'epochs': 5
}

In [7]:
# params = {
#     'input_dim': input_size,             # Должно быть задано в соответствии с данными
#     'share_dim': [2], # Варианты размерности общих слоев
#     'base_dim': [2],   # Варианты размерности базовых слоев
#     'do_rate': [0.1, 0.2, 0.3],  # Варианты dropout
#     'batch_norm': [True, False], # Использование BatchNorm
#     'normalization': ['none', 'divide'], # Тип нормализации
#     'factual_loss_weight': [0.8, 1.0, 1.2], # Вес фактической потери
#     'propensity_loss_weight': [0.05, 0.1, 0.2], # Вес потери пропенсити
#     'tau_loss_weight': [0.05, 0.1, 0.2],    # Вес потери tau (если применимо)
#     'batch_size': 32,
#     'gradient_accumulation_steps' : 2,
#     'epochs': 1
# }

In [40]:
configs = DESCNUpliftModel.generate_config(count=4, params = params)

In [41]:
factory = DESCNUpliftModelFactory
config = configs[0]
batch_size=32
max_size=100000

In [42]:
model, train, test = factory.create(config, train_path, test_path)

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
# from torch.utils.data import DataLoader
# dl = DataLoader(
#     train, 
#     batch_size=32, 
#     shuffle=True,

#     num_workers=0
# )

In [12]:
# kek = model.predict(train)

In [13]:
# pd.DataFrame(kek['uplift'])

In [14]:
# q = model.predict(train)

In [15]:
# q['df'][q['df']['treatment'] == 1]

In [43]:
train_test_model(ds_name='lazada_v2', features_percent=100, factory=DESCNUpliftModelFactory, config=configs[0], NN=True)   

/home/ogrobertino/test_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1/5 - 10.0% - Loss: 0.8167
Epoch 1/5 - 20.0% - Loss: 0.7829
Epoch 1/5 - 30.0% - Loss: 0.7774
Epoch 1/5 - 40.0% - Loss: 0.7629
Epoch 1/5 - 50.0% - Loss: 0.7509
Epoch 1/5 - 60.0% - Loss: 0.7460
Epoch 1/5 - 70.0% - Loss: 0.7385
Epoch 1/5 - 80.0% - Loss: 0.7319
Epoch 1/5 - 90.0% - Loss: 0.7277
Epoch 1/5 - 100.0% - Loss: 0.7259
Validation after epoch
Epoch 1/5, Train Loss: 0.7259, Val Loss: 0.6688, Val AUUC: -0.0628, LR: 0.001000
Epoch 2/5 - 10.0% - Loss: 0.6831
Epoch 2/5 - 20.0% - Loss: 0.6794
Epoch 2/5 - 30.0% - Loss: 0.6800
Epoch 2/5 - 40.0% - Loss: 0.6844
Epoch 2/5 - 50.0% - Loss: 0.6820
Epoch 2/5 - 60.0% - Loss: 0.6811
Epoch 2/5 - 70.0% - Loss: 0.6832
Epoch 2/5 - 80.0% - Loss: 0.6832
Epoch 2/5 - 90.0% - Loss: 0.6837
Epoch 2/5 - 100.0% - Loss: 0.6840
Validation after epoch
Epoch 2/5, Train Loss: 0.6840, Val Loss: 0.6721, Val AUUC: -0.0548, LR: 0.001000
Epoch 3/5 - 10.0% - Loss: 0.6717
Epoch 3/5 - 20.0% - Loss: 0.6758
Epoch 3/5 - 30.0% - Loss: 0.6789
Epoch 3/5 - 40.0% - Loss: 0.677

cfg:{'lr': 0.001, 'decay_rate': 0.95, 'decay_step_size': 1, 'l2': 0.001, 'model_name': 'DESCN_128_64_20220215_175927', 'n_experiments': 5, 'batch_size': 5000, 'share_dim': 128, 'base_dim': 64, 'reweight_sample': 1, 'val_rate': 0.2, 'do_rate': 0.1, 'normalization': 'divide', 'epochs': 5, 'log_step': 50, 'pred_step': 1, 'optim': 'Adam', 'BatchNorm1d': 'true', 'prpsy_w': 0.5, 'escvr1_w': 0.5, 'escvr0_w': 1, 'h1_w': 0, 'h0_w': 0, 'mu0hat_w': 0.5, 'mu1hat_w': 1, 'imb_dist': 'wass', 'imb_dist_w': 0,

In [ ]:
# for i in tqdm(range(len(configs)), desc="Training models"):
#     train_test_model(ds_name='lazada', features_percent=100, factory=DRModelFactory, config=configs[i])   

In [ ]:
from src.global_params import BASE_PATH
path = pd.read_csv(BASE_PATH + "/exps2/stats.tsv", sep='\t').Path.iloc[-1]
model = DESCNUpliftModel(from_load=True, path = str(BASE_PATH) + "/" + path)

In [ ]:
model.config